In [1]:
import pandas as pd
import numpy as np
import  matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
df_trash = pd.read_csv('../data/trash_hauler_report_with_lat_lng.csv')
df_trash.head();

In [3]:
df_trash['Trash Hauler'].unique()

array(['RED RIVER', 'WASTE IND', 'METRO', nan, 'Metro'], dtype=object)

In [4]:
# Filters only 'RED RIVER' Trash Hauler
df_red_river = df_trash[df_trash['Trash Hauler'] == 'RED RIVER']
df_red_river.head();

In [5]:
df_trash['Request'].value_counts();

In [6]:
df_missed_pickup = df_red_river[df_red_river['Request'].isin(['Trash - Curbside/Alley Missed Pickup', 'Trash - Backdoor', 'Trash Collection Complaint'])]
df_missed_pickup.head();

In [7]:
df_missed_pickup.shape

(14198, 13)

In [8]:
df_value_count = df_missed_pickup[['Trash Hauler','Incident Address']].value_counts().reset_index()
df_value_count.head();

In [9]:
df_missed_2plus = df_value_count[df_value_count['count']>1]
df_missed_2plus['fine']= (df_missed_2plus['count']-1) *200
#df_missed_2plus.head()
df_missed_2plus.groupby('Trash Hauler')['fine'].sum();


C:\Users\jinor\AppData\Local\Temp\ipykernel_8168\630706056.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_missed_2plus['fine']= (df_missed_2plus['count']-1) *200


In [10]:
df_trash['Request'].unique()

array(['Trash - Backdoor', 'Trash - Curbside/Alley Missed Pickup',
       'Trash Collection Complaint', 'Damage to Property'], dtype=object)

In [11]:
keywords = df_trash['Description'].str.contains(r'missed|pickup|alley|miss', case=False, na=False, regex=True)
print(f"Found {keywords.sum()} rows with any of these words")
df_missed = df_trash[keywords][['Request Number','Trash Hauler','Description','Incident Address','Zip Code','Trash Route']]
df_missed.head();
#df_missed.shape

Found 14293 rows with any of these words


In [12]:
df_performance = df_missed.groupby('Trash Hauler').agg(total_request= ('Request Number', 'count'),total_routes=('Trash Route','nunique') )
df_performance;

## ⦁	What were to total missed pickup by route?

In [13]:
df_missed.head()

,Request Number,Trash Hauler,Description,Incident Address,Zip Code,Trash Route
1,25274,RED RIVER,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,4202
2,25276,RED RIVER,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,4205
3,25307,WASTE IND,missed,2603 old matthews rd,37207.0,2206
4,25312,RED RIVER,Missed the even side of the road.,604 croley dr,37209.0,4203
5,25317,RED RIVER,left trash cart in middle of driveway instead ...,3602 floral dr,37211.0,4304


In [14]:
df_route_counts = df_missed.groupby(['Trash Hauler','Trash Route'])['Request Number'].count().reset_index().sort_values(by='Request Number', ascending= False)
df_route_counts.head()

,Trash Hauler,Trash Route,Request Number
155,RED RIVER,4504,273
100,RED RIVER,3302,231
12,METRO,9303,218
44,RED RIVER,1303,209
148,RED RIVER,4404,209
